In [1]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle
import jdatetime
#starting date = 29 tir

In [3]:
df = pd.read_csv("data.csv")

C:\Users\Arian_Gh\AppData\Local\Temp\ipykernel_13552\3477112522.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


In [5]:
df["company_size"].unique()

array(['11 تا 50 نفر', '51 تا 200 نفر', 'بیش از 5000 نفر',
       '501 تا 1000 نفر', '201 تا 500 نفر', '1001 تا 5000 نفر',
       'زیر 10 نفر', '11 - 50 employees', '501 - 1000 employees', nan,
       '1001 - 5000 employees', '51 - 200 employees',
       '201 - 500 employees', 'More than 5001 employees',
       'Less than 10 employees'], dtype=object)

In [6]:
df.columns

Index(['company_size', 'education', 'gender', 'salary', 'workdays', 'language',
       'business_trip', 'age', 'industry', 'bonus', 'location', 'ad_name',
       'softwares', 'military_services', 'experience', 'date'],
      dtype='object')

In [ ]:
# A dictionary to map Persian/English categories to their midpoints
size_mapping = {
    '11 تا 50 نفر': '30', 
    '51 تا 200 نفر': '125', 
    'بیش از 5000 نفر': '5000+', 
    '501 تا 1000 نفر': '750', 
    '201 تا 500 نفر': '350', 
    '1001 تا 5000 نفر': '3000', 
    'زیر 10 نفر': '<10', 
    '11 - 50 employees': '30', 
    '501 - 1000 employees': '750', 
    '1001 - 5000 employees': '3000', 
    '51 - 200 employees': '125', 
    '201 - 500 employees': '350', 
    'More than 5001 employees': '5000+', 
    'Less than 10 employees': '<10'
}

# Replace the values using the map
df['company_size'] = df['company_size'].map(size_mapping)

# For NaN values, you could either fill them with a placeholder like 'Unknown' or you could leave them as NaN if you wish to represent missing data
df['company_size'].fillna('Unknown', inplace=True)